# Example case of estimating observing time for assumed A10 clusters #

This thread shows how to use _M2_ProposalTools_ to estimate the required time to observe clusters based on a single pointing.

In [1]:
import WorkHorse as WH
import numpy as np
import astropy.units as u
import FilterImages as FI
import scipy

In [2]:
from importlib import reload

In [3]:
WH=reload(WH)

Let's just take the case of a few masses at high redshift ($z=1.5$)

In [4]:
M500s = np.array([3.0,4.0,5.0,6.0])*1e14*u.M_sun
zs    = np.array([1.5,1.5,1.5,1.5]) 

In [7]:
h70=1
fwhm           = 10.0
pixsize        = 2.0
s2f            = np.sqrt(8.0*np.log(2.0)) 
pix_sigma      = fwhm/(pixsize*s2f)
#xferfile       = "/home/data/MUSTANG2/IDL_Xfer_Fxns/ByScanSize/pca5_0f08/xfer_Function_3p0_21Aonly_PCA5_0f08Filtering.txt"
xferfile       = "xfer_Function_3p0_21Aonly_PCA5_0f08Filtering.txt"
y2k            = -3.3

In [8]:
TotalTime=0
for M500,z in zip(M500s,zs):

    y500       = WH.y_delta_from_mdelta(M500.value,z,delta=500,YMrel="A10",ycyl=False,h70=h70)
    
    ymap       = WH.make_A10Map(M500,z,pixsize=pixsize,Dist=True)
    mymap      = WH.smooth_by_M2_beam(ymap,pixsize=pixsize)
    
    tab        = FI.get_xfer(xferfile)
    yxfer_init = FI.apply_xfer(mymap,tab,pixsize)
    yxfer      = scipy.ndimage.filters.gaussian_filter(yxfer_init, pix_sigma*0.9)
    uKmap      = yxfer*y2k*1e6
    uJymap     = uKmap * (56/73.0)
    MS         = 56.2
    
    #print("pixel size: ",pixsize)
    SZpeak_uK  = np.min(uKmap)
    SZpeak_uJy = np.min(uJymap)
    SZpeak_yr  = np.max(ymap)
    SZpeak_yc  = np.max(mymap)
    SZpeak_yx  = np.max(yxfer)
    ObsTime    = (MS*10/SZpeak_uJy)**2
    TelTime    = np.ceil(ObsTime*8)/4.0
    print("M500 (1e14): ",(M500/1e14).value)
    print("Redshift: ",z)
    #print("R500 (arcmin): ",Theta500*60*180/np.pi)
    print("Y500 (1e12 sr): ",y500*1e12)
    print("Unfiltered y peak: ", SZpeak_yr)
    print("Beam-convolved y peak: ", SZpeak_yc)
    print("Filtered y peak: ", SZpeak_yx)
    print("Filtered peak, uK: ", SZpeak_uK)
    print("Filtered peak, uJy/beam: ", SZpeak_uJy)
    print("Time to 10 sigma peak detection (hrs): ",ObsTime)
    print("================================================")
    TotalTime += TelTime
    
print("Total Telescope Time: ", TotalTime)

M500 (1e14):  4.9
Redshift:  1.86
Y500 (1e12 sr):  28.173524535508903
Unfiltered y peak:  0.0004035995631544672
Beam-convolved y peak:  0.0003680112181241709
Filtered y peak:  0.0001346494401576201
Filtered peak, uK:  -444.3431525201463
Filtered peak, uJy/beam:  -340.8659800154547
Time to 10 sigma peak detection (hrs):  2.718349625131644
M500 (1e14):  4.2
Redshift:  1.67
Y500 (1e12 sr):  19.92521636575886
Unfiltered y peak:  0.00028267246311782624
Beam-convolved y peak:  0.00025634591675205274
Filtered y peak:  9.332350033556027e-05
Filtered peak, uK:  -307.96755110734887
Filtered peak, uJy/beam:  -236.2490803015279
Time to 10 sigma peak detection (hrs):  5.658904673259357
M500 (1e14):  4.4
Redshift:  1.63
Y500 (1e12 sr):  21.34256014825589
Unfiltered y peak:  0.0002869534254484513
Beam-convolved y peak:  0.00026240870661402
Filtered y peak:  9.432950908470593e-05
Filtered peak, uK:  -311.28737997952953
Filtered peak, uJy/beam:  -238.795798340461
Time to 10 sigma peak detection (hrs): 